In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D

In [3]:
od.download('https://www.kaggle.com/datasets/arunrk7/surface-crack-detection')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: mangveer31
Your Kaggle Key: ··········


100%|██████████| 233M/233M [00:01<00:00, 127MB/s]


In [4]:
images=[]
labels=[]

In [5]:
postive_dataset_bath = '/content/surface-crack-detection/Positive'

In [6]:
images=[]
labels=[]
for file in os.listdir(postive_dataset_bath):
  img_path = os.path.join(postive_dataset_bath,file)
  img = cv2.imread(img_path)
  img = cv2.resize(img,(64,64))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  images.append(img)
  labels.append(1)
  print(img)



Streaming output truncated to the last 5000 lines.
  [206 197 188]
  [205 196 187]]]
[[[181 177 165]
  [182 178 166]
  [185 181 169]
  ...
  [182 178 166]
  [183 179 167]
  [180 176 164]]

 [[177 173 160]
  [182 178 167]
  [180 176 165]
  ...
  [177 173 161]
  [176 172 160]
  [178 174 162]]

 [[175 171 159]
  [185 181 169]
  [183 179 167]
  ...
  [180 176 164]
  [177 173 162]
  [179 175 163]]

 ...

 [[183 181 169]
  [191 189 177]
  [178 176 164]
  ...
  [187 179 166]
  [191 183 170]
  [196 192 177]]

 [[186 184 172]
  [183 181 169]
  [181 179 167]
  ...
  [194 186 173]
  [191 183 170]
  [192 188 173]]

 [[183 180 169]
  [184 180 169]
  [182 179 168]
  ...
  [196 188 175]
  [198 190 177]
  [190 185 173]]]
[[[178 174 162]
  [170 166 154]
  [164 160 148]
  ...
  [103 100  91]
  [126 122 113]
  [213 211 199]]

 [[176 172 160]
  [173 169 157]
  [168 164 153]
  ...
  [ 57  54  47]
  [ 63  60  53]
  [140 137 128]]

 [[174 170 158]
  [172 168 156]
  [169 165 153]
  ...
  [ 49  48  43]
  [ 47 

In [7]:
negative_dataset_path= '/content/surface-crack-detection/Negative'

In [8]:
images=[]
labels=[]
for file in os.listdir(negative_dataset_path):
  img_path = os.path.join(negative_dataset_path,file)   # negative_dataset_path + file ==> we can read that image
  img = cv2.imread(img_path) #reading img
  img = cv2.resize(img,(64,64))#all images of one size
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)#color scheme
  images.append(img)
  labels.append(0)

In [9]:
images = np.array(images)
labels = np.array(labels)

In [10]:
images.shape

(20000, 64, 64, 3)

In [11]:
xtrain,xtest,ytrain,ytest = train_test_split(images,labels, test_size =0.2)

In [12]:
#max pixel value is 255 and we need in 0 to 1 hence converting to 0 to 1
xtrain = xtrain/255.0
xtest = xtest/255.0

In [13]:
model = Sequential()

In [14]:
model.add(Conv2D(32,(3,3), activation = 'relu', input_shape = (64,64,3)))

In [15]:
model.add(MaxPooling2D(2,2))

In [16]:
model.add(Conv2D(64,(3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))

In [17]:
model.add(Conv2D(64,(3,3), activation = 'relu'))

In [18]:
model.add(Flatten())

In [19]:
model.add(Dense(64, activation='relu'))

In [20]:
model.add(Dense(1, activation ='sigmoid'))

In [21]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [22]:
model.fit(xtrain,ytrain, epochs = 2, batch_size = 32, validation_data = (xtest,ytest))

Epoch 1/2
500/500 [==============================] - 91s 178ms/step - loss: 0.0017 - accuracy: 0.9981 - val_loss: 6.4707e-22 - val_accuracy: 1.0000
Epoch 2/2
500/500 [==============================] - 88s 175ms/step - loss: 5.2048e-22 - accuracy: 1.0000 - val_loss: 6.4707e-22 - val_accuracy: 1.0000


In [23]:
loss, accuracy = model.evaluate(xtest,ytest)
print(loss, accuracy)

125/125 [==============================] - 8s 60ms/step - loss: 6.4707e-22 - accuracy: 1.0000
6.4706950626010015e-22 1.0


In [25]:
import cv2
import numpy as np

image_path = "/content/images (1).jpeg"

# Check if the image is loaded successfully
image = cv2.imread(image_path)
if image is None:
    print("Error: Image could not be loaded")
    exit()

image = cv2.resize(image, (64, 64))
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = np.expand_dims(image, axis=0)
image.shape
image = image / 255.0


In [26]:
model.predict(image)

1/1 [==============================] - 0s 168ms/step


array([[1.6517014e-27]], dtype=float32)

In [27]:
output = model.predict(image)
print("Crack Detected") if output[0][0] > 0.5 else print("No crack Detected")

1/1 [==============================] - 0s 39ms/step
No crack Detected
